In [1]:
from src.helpers import DBConnector
import pandas as pd

In [2]:
db = DBConnector('/Users/ignasi/Documents/_03_MDS_/_01_ADSDB_/ui/simulation/trusted.duckdb')

In [3]:
db.get_tables()

['mvc_crash', 'mvc_person', 'mvc_vehicles', 'nhtsa_safety_rating']

In this exploitation layer we would like "person_id", "person_age", "person_sex", "vehicle_make", "vehicle_year", "overall_rating". As we would like to be able to analyze which type of vehicles does people of certain age buy, and see if there exists any type of correlation between the vehicle and the probabilty of having and accident. 

In [4]:
mvc_crash = db.get_table_as_dataframe("mvc_crash")
mvc_person = db.get_table_as_dataframe("mvc_person")
mvc_vehicles = db.get_table_as_dataframe("mvc_vehicles")
nhtsa_safety_rating = db.get_table_as_dataframe("nhtsa_safety_rating")

In [8]:
filtered_person = mvc_person[mvc_person["person_type_occupant"]][["person_id", "collision_id", "person_age", "person_sex"]]
filtered_person.head()

,person_id,collision_id,person_age,person_sex
0,4e11033f-631f-4c1e-8e35-25f364771e47,4692029,34,M
1,134f4db1-5941-4e3f-9425-896ba303cf61,4691811,24,M
2,b6b48952-f85d-4d75-a74e-02ad06c47ea9,4692055,36,F
3,57d9fca7-772f-4838-ac5d-4a4573d605de,4691840,39,U
5,720a3744-bb6f-4d3f-a29c-f7a376b8cb23,4692055,71,F


In [11]:
person_vehicle = filtered_person.merge(
    mvc_vehicles,
    how="left",
    left_on="collision_id",
    right_on="collision_id",
)
person_vehicle.head()

,person_id,collision_id,person_age,person_sex,index,vehicle_id,vehicle_type,vehicle_make,vehicle_model,vehicle_year,...,overturned,right_front_bumper,right_front_quarter_panel,right_rear_bumper,right_rear_quarter_panel,right_side_doors,roof,trailer,undercarriage,crash_datetime
0,4e11033f-631f-4c1e-8e35-25f364771e47,4692029,34,M,399,5f37fb5b-3313-4756-b883-8f8788314656,pickup truck,alfa romeo,NaN,2019.0,...,0,0,0,0,0,0,0,0,0,2024-01-01 16:00:00
1,134f4db1-5941-4e3f-9425-896ba303cf61,4691811,24,M,167,0fd612fb-7145-480f-854d-611838975154,station wagon,alfa romeo,NaN,2005.0,...,0,0,0,1,0,0,0,0,0,2024-01-01 11:25:00
2,b6b48952-f85d-4d75-a74e-02ad06c47ea9,4692055,36,F,45,702f4bbb-4f42-414c-b8ff-66a18acbfed3,pickup truck,ford,NaN,2013.0,...,0,1,0,0,0,0,0,0,0,2024-01-01 17:37:00
3,57d9fca7-772f-4838-ac5d-4a4573d605de,4691840,39,U,14,22200b9b-4313-42b5-841d-6af0bf309fd9,sedan,ford,NaN,2008.0,...,0,0,1,0,0,1,0,0,0,2024-01-01 10:15:00
4,720a3744-bb6f-4d3f-a29c-f7a376b8cb23,4692055,71,F,45,702f4bbb-4f42-414c-b8ff-66a18acbfed3,pickup truck,ford,NaN,2013.0,...,0,1,0,0,0,0,0,0,0,2024-01-01 17:37:00


In [12]:
person_vehicle = person_vehicle[
                [
                    "person_id",
                    "person_age",
                    "person_sex",
                    "vehicle_make",
                    "vehicle_year",
                ]
            ]

As we are working with samples of the differents datasets, at this stage, is normal to have null values. We can assert that in future iterations, when more computational power is able, we could work with the full datasets and extract this tables without null values.

In [13]:
person_vehicle.merge(
    nhtsa_safety_rating,
    how="inner",
    left_on=["vehicle_make", "vehicle_year"],
    right_on=["make", "model_year"],
)[
    [
        "person_id",
        "person_age",
        "person_sex",
        "vehicle_make",
        "vehicle_year",
        "overall_rating",
    ]
]

,person_id,person_age,person_sex,vehicle_make,vehicle_year,overall_rating
0,4e11033f-631f-4c1e-8e35-25f364771e47,34,M,alfa romeo,2019.0,NaN
1,4e11033f-631f-4c1e-8e35-25f364771e47,34,M,alfa romeo,2019.0,NaN
2,4e11033f-631f-4c1e-8e35-25f364771e47,34,M,alfa romeo,2019.0,NaN
3,4e11033f-631f-4c1e-8e35-25f364771e47,34,M,alfa romeo,2019.0,NaN
4,4e11033f-631f-4c1e-8e35-25f364771e47,34,M,alfa romeo,2019.0,NaN
...,...,...,...,...,...,...
26583,9964f635-78c6-4b3c-a683-1d1b8abbce5f,29,M,gmc,2002.0,3.500000
26584,9964f635-78c6-4b3c-a683-1d1b8abbce5f,29,M,gmc,2002.0,2.750000
26585,9964f635-78c6-4b3c-a683-1d1b8abbce5f,29,M,gmc,2002.0,3.000000
26586,9964f635-78c6-4b3c-a683-1d1b8abbce5f,29,M,gmc,2002.0,3.666667
